# Identifying Edible Plants
## w251 Final Project
## Scott Xu, Divya Babu, Aaron Olson

The Intent of this final project is to develop an image recognition program that can accurately identify edible and/or poisonous plants in the wild. This endeavor has been attempted by several apps and other programs - however all of these realize an edge architecture that relies on a remote server connection in order to upload the file and run through the model. 

This paper explores the difference performance options in order to arrive at the best performing model. We then work to reduce the model size in order to fit on an edge device for real time diagnosis. 

In order to get a baseline model for image recognition, we used a transfer learning technique where the model weights and architecture of ResNet50 was applied. ResNet50 was chosen for its performance as well as its size. Training on the volume of images for the duration that ResNet50 was done would not be reasonable - therefore we have used this baseline model to improve the baseline prediction. On top of this we explore different model architectures in order to define which architecture performs the best.

In order to get the best performing model we needed to remember to balance model performance with edge device performance. In the case of poisonous plants the consequences of a bad prediction can be high - however the utility of an app that takes 60 min to make a prediction is impacted. Therefore at the end of this notebook we examine the relationship with building the model on a virtual machine (for training) vs inference on the edge device (time to predict vs accuracy). 

We begin by examining the training dataset:

In [ ]:
# TODO: Insert cells with example pictures / labels and a few details about training size / etc

In [2]:
# Load ResNet50 baseline model
from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 300
WIDTH = 300

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


### Image Augmentation

With the baseline model established, we understand that there will likely be a difference in the cleanliness of the images taken for the training dataset, vs the images taken in the field when a user wants to succesfully identify a plant. We therefore utilize image augmentation, to achieve a couple of tasks: (1) affect the image quality, orientation, etc in order to make the model more versatile (2) create more training images in order to train the model. 

Below we explore different effects of image augmentation and show below the effects of model performance: 

In [3]:
# Image augmentation

from keras.preprocessing.image import ImageDataGenerator
import os
from glob import glob

TRAIN_DIR = "train_edible"
result = [y for x in os.walk(TRAIN_DIR) for y in glob(os.path.join(x[0], '*.jpg'))]
classes = list(set([y.split("/")[-2] for y in result]))
HEIGHT = 300
WIDTH = 300
BATCH_SIZE = 32

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True,
      height_shift_range=0.5,
      width_shift_range=0.5
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)

Found 4220 images belonging to 37 classes.


### Model Architecture - Predicting Plant Class

Building on the baseline model, we have explored adding different layers (size, type, etc) in order to produce the 'best' performing model. See above and later discussions as to how we define 'best' model. For this paper we explored both accurately predicting the class of the plant in an image - which would help users understand more information about the specific plant they have taken the picture of, as well as predicting whether a plant is poisonous or not without the context of exact plant type. 

The original intent of this paper was to predict the plant class whereby the metadata associated with that prediction would yield the label of poisonous or edible - however we also chose to explore whether deep learning could accurately predict the binary choice of poisonous or not. 

In [4]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # Can look here if adding different types of layers has an effect
        # Also explore differences in changing activation function
        # Can also iterate on droupout amount
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

# Can change the model architecture here
FC_LAYERS = [128,64,32,16]
dropout = 0.3

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(classes))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import datetime
import numpy   
    
# For the baseline model will use 100 epochs and 15000 images to test model performance
# Will then use 'optimized' model parameters to train for longer time and explore
# Size vs Accuracy for edge compute purposes
NUM_EPOCHS = 100
BATCH_SIZE = 64
num_train_images = 50000

adam = Adam(lr=0.00001)
# Can look into whether 
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])
# Checkpoin is overwritten at each epoch - can look at line below where datetime is used to create time based file names
filepath="checkpoint/" + "edible_default" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
# log_dir = "C:\\Users\\AOlson\\Documents\\UC Berkeley MIDS\\w251_Scaling\\Final Project\\Data\\log_dir\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True, callbacks=callbacks_list)

numpy.savetxt('edible_loss_history.txt', numpy.array(history.history['loss']), delimiter = ',')
numpy.savetxt('edible_acc_history.txt', numpy.array(history.history['acc']), delimiter = ',')

Epoch 1/100
781/781 [==============================] - 123s 158ms/step - loss: 3.2889 - acc: 0.1154

Epoch 00001: saving model to checkpoint/edible_default_model_weights.h5
Epoch 2/100
781/781 [==============================] - 120s 154ms/step - loss: 3.1407 - acc: 0.1662

Epoch 00002: saving model to checkpoint/edible_default_model_weights.h5
Epoch 3/100
781/781 [==============================] - 120s 153ms/step - loss: 3.0145 - acc: 0.2026

Epoch 00003: saving model to checkpoint/edible_default_model_weights.h5
Epoch 4/100
781/781 [==============================] - 120s 154ms/step - loss: 2.9042 - acc: 0.2331

Epoch 00004: saving model to checkpoint/edible_default_model_weights.h5
Epoch 5/100
781/781 [==============================] - 120s 154ms/step - loss: 2.8408 - acc: 0.2521

Epoch 00005: saving model to checkpoint/edible_default_model_weights.h5
Epoch 6/100
781/781 [==============================] - 120s 153ms/step - loss: 2.7598 - acc: 0.2763

Epoch 00006: saving model to check